### Práctica de Web Scraping

Se va a considerar una página web de wikipedia donde se encuentra un recopilatorio de las pinturas realizadas por Francisco Goya: https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya

En esta página aparece una tabla en la que aparece cada pintura junto a información asociada a la pintura.

In [ ]:
from IPython.display import Image
Image(filename='Captura.png')

La tabla está formada por 6 columnas: Cuadro, Título, Fecha, Dimensiones, Museo y País.
Observa cómo es la estructura de una fila

Los enlaces a las imágenes se encuentran en el atributo src del elemento <img>. En el ejemplo anterior, el valor de src es:
//upload.wikimedia.org/wikipedia/commons/thumb/f/f1/El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg/150px-El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg

Para acceder a la imagen se debe añadir https: delante del valor recuperado:

https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg/150px-El_ap%C3%B3stol_Santiago_y_sus_disc%C3%ADpulos_adorando_a_la_Virgen_del_Pilar.jpg

Para recuperar el contenido de la página html de wikipedia, se puede hacer de la siguiente manera:

In [ ]:
import requests
url="https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya"
r = requests.get(url)
html = r.text
print(html)

Se pide hacer un buscador que permita recuperar información de la página. Para ello se le mostrará al usuario dos opciones entre las que tiene que elegir: Buscar por país o Buscar por museo.

__Buscar por país[5 puntos] __
 
Si elige esta opción se le mostrará un listado de todos los paises en los que existe una pintura de Goya. Para facilitar la selección se asignará un número a cada páis, de manera que el usuario elija por número. Los países se pueden recuperar de la última columna de la tabla de la página de wikipedia. Una vez elegido el país, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y Museo. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre país.

__Buscar por museo[5 puntos] __
 
Si elige esta opción se le mostrará un listado de todos los museos en los que existe una pintura de Goya. Para facilitar la selección se asignará un número a cada museo, de manera que el usuario elija por número. Los museos se pueden recuperar de la penúltima columna de la tabla de la página de wikipedia. Una vez elegido el museo, el programa mostrará una lista de todas las pinturas que cumplen ese requisito. De cada pintura se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y País. Así mismo, se creará una carpeta en el sistema de archivos, y se guardarán todas las imagenes de las pinturas que cumplen la condición de búsqueda sobre museo.

__Nota: Para procesar la página html es obligatorio utilizar la libreria BeautifulSoup. No se pueden usar otras librerías no vistas en clase__

In [10]:
#Elige opcion
option = menu()

import urllib
import requests
from bs4 import BeautifulSoup
url = "https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya"
r = requests.get(url)
html = r.text

soup = BeautifulSoup(html, 'html.parser')

if int(option) == 1:
    buscarPorPais(soup)
else:
    buscarPorMuseo(soup)
    

MENÚ
----
1 - Buscar por país
2 - Buscar por museo
----
Introduce elección: 2

1  -  San Juan El Real (Calatayud) 
2  -  Museo de Zaragoza (desde 1985) procedente de San Juan El Real de Calatayud (1763?-1767) y de la ermita de la Virgen de Jaraba (1767-1985) 
3  -  Colección privada (Aragón) 
4  -  Colección Marquesa de las Palmas (Jerez de la Frontera) 
5  -  Fundación Selgas-Fagalde (Cudillero) 
6  -  Museo de Zaragoza 
7  -  Colección particular 
8  -  Colección José Gudiol (Barcelona) 
9  -  Colección Félix Palacios Remondo (Zaragoza) 
10 -  Colección privada 
11 -  Museo Provincial, Zaragoza 
12 -  colección privada 
13 -  Basílica de Nuestra Señora del Pilar (Zaragoza) 
14 -  Museo del Prado (Madrid) 
15 -  colección privada (Barcelona)[7]​ 
16 -  Cartuja de Aula Dei (Zaragoza) 
17 -  Colección privada, Zaragoza 
18 -  Museo Goya 
19 -  Museo de Huesca 
20 -  Meadows Museum (Dallas) 
21 -  Real Academia de Bellas Artes San Carlos 
22 -  Banco de España 
23 -  Museo Zuloaga 
24 - 

In [1]:
def menu():
    print("MENÚ")
    print("----")
    print("1 - Buscar por país")
    print("2 - Buscar por museo")
    print("----")
    inputCorrecto = False
    
    while not inputCorrecto:
        elemento = input("Introduce elección: ")
        try:
            if int(elemento) > 0 and int(elemento) < 3:
                inputCorrecto = True
            else:
                print("\nError. Introduce una opción posible.")
        except:
            print("\nError. Introduce un numero correcto.") # Necesitamos numeros
    return elemento

In [2]:
#Busqueda por pais
def buscarPorPais(soup):
    
    etiquetas = soup("tr")
    print()
    listaSrc = []
    paisDelSrc = {}
    lista = listaPaises(etiquetas, listaSrc, paisDelSrc)

    # Sacamos por pantalla los paises
    i = 1
    for pais in lista:
        if i > 9:
            print(i , "- ", pais)
        else:
            print(i , " - ", pais)
        i = i + 1
        
    #Elige un pais
    elemento = inputUsuario(len(lista))
            
    pinturas = cuadrosPais(lista[int(elemento) - 1], etiquetas)
    
    #Guardo imagenes en carpeta
    guardaCarpeta(lista[int(elemento) - 1], etiquetas, listaSrc, paisDelSrc)

In [3]:
def listaPaises(etiquetas, listaSrc, paisDelSrc):
    lista = []
    i = 0
    for etiqueta in etiquetas: 
        if i > 0:
            s = 0
            for item in etiqueta:
                if s == 1:
                    pos = str(item).find("src=")
                    elem = str(item)[pos + 5:]
                    pos = elem.find("srcset")
                    elem = str(elem)[:pos - 2]
                    elem = "https:" + elem
                    listaSrc.append(elem)
                if s == 11:
                    newPais = item.text.split()

                    if len(newPais) > 0:
                        aux = newPais[0]
                        if newPais[0] == "República": #Es el único que no está duplicado
                            aux = aux  + " " + newPais[1]
                        if len(newPais) > 2:
                            aux = ""
                            j = 0
                            for member in newPais:
                                if j < len(newPais) / 2:
                                    aux = aux + member + " "
                                j = j + 1
                        paisDelSrc[listaSrc[len(listaSrc)  - 1]] = aux
                        if aux not in lista:   
                            lista.append(aux)
                    else:
                        paisDelSrc[listaSrc[len(listaSrc)  - 1]] = "NADA"
                s = s + 1
        i = i + 1
    return lista

In [4]:
def cuadrosPais(pais, etiquetas):
    #Se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y Museo
    lista = []
    i = 0
    for etiqueta in etiquetas:
        if i > 0:
            s = 0
            for item in etiqueta:
                if pais[int(len(pais) - 1)] == " ":
                    pais = pais[:int(len(pais) - 1)]
                if s == 11 and len(item.text) > 0 and item.text[:int(len(item.text) / 2 - 1)] == pais:
                    lista.append(etiqueta)
                s = s + 1
        i = i + 1
    
    for etiqueta in lista:
        a = etiqueta.text[:len(etiqueta.text) - 2 * len(pais) - 2].strip()
        print(a)
        print("_____________________")
        
    return True

In [5]:
import os
import urllib.request
def guardaCarpeta(pais, etiquetas, listaSrc, paisDelSrc):

    path = input("¿En qué carpeta guardas las imagenes? ")
    if not os.path.exists(path):
        try:
            os.makedirs(path)
        except OSError:
            print("Error creatng the folder!")

    os.chdir(path)
    
    #Guardar imagenes
    x = 1
    for url in listaSrc:
        if paisDelSrc[url] == pais:
            img = urllib.request.urlopen(url)
            name = "imagen" + str(x) + ".jpg"
            f = open(name,"wb")
            while True:
                info = img.read(100000)
                if len(info) < 1: 
                    break
                f.write(info)
            f.close()
            x = x + 1
    
    print("Imagenes guardadas correctamente!")

In [6]:
def inputUsuario(leng):
    inputCorrecto = False
    while not inputCorrecto:
        elemento = input("Introduce elección: ")
        try:
            if int(elemento) > 0 and int(elemento) < leng + 1:
                inputCorrecto = True
            else:
                print("\nError. Introduce una opción posible.")
        except:
            print("\nError. Introduce un numero correcto.") # Necesitamos numeros
    return elemento

In [7]:
#Busqueda por museo
def buscarPorMuseo(soup):
   
    etiquetas = soup("tr")
    print()
    listaSrc = []
    museoDelSrc = {}
    lista = listaMuseos(etiquetas, listaSrc, museoDelSrc)

    # Sacamos por pantalla los paises
    i = 1
    for pais in lista:
        if i > 9:
            print(i , "- ", pais)
        else:
            print(i , " - ", pais)
        i = i + 1
        
    #Elige un pais
    elemento = inputUsuario(len(lista))
            
    pinturas = cuadrosMuseo(lista[int(elemento) - 1], etiquetas) #TODO 
    
    #Guardo imagenes en carpeta
    guardaCarpeta(lista[int(elemento) - 1], etiquetas, listaSrc, museoDelSrc)

In [8]:
def listaMuseos(etiquetas, listaSrc, museoDelSrc):
    lista = []
    i = 0
    for etiqueta in etiquetas: 
        if i > 0:
            s = 0
            for item in etiqueta:
                if s == 1:
                    pos = str(item).find("src=")
                    elem = str(item)[pos + 5:]
                    pos = elem.find("srcset")
                    elem = str(elem)[:pos - 2]
                    elem = "https:" + elem
                    listaSrc.append(elem)
                if s == 9:
                    newMuseo = item.text.split()

                    aux = ""
                    if len(newMuseo) > 0:
                        for item in newMuseo:
                            aux =  aux + item + " "
                        museoDelSrc[listaSrc[len(listaSrc)  - 1]] = aux 
            
                        if aux not in lista:   
                            lista.append(aux)
                    else:
                        museoDelSrc[listaSrc[len(listaSrc)  - 1]] = "NADA"
                s = s + 1
        i = i + 1
    return lista

In [9]:
def cuadrosMuseo(museo, etiquetas):
    #Se mostrará la información que aparece en las columnas: Título, Fecha, Dimensiones, y Museo
    lista = []
    i = 0
    for etiqueta in etiquetas:
        if i > 0:
            s = 0
            for item in etiqueta:
                if museo[int(len(museo) - 1)] == " ":
                    museo = museo[:int(len(museo) - 1)]
                if s == 9 and len(item.text) > 0 and (item.text == museo or item.text[:int(len(item.text) - 1)] == museo):
                    lista.append(etiqueta)
                s = s + 1
        i = i + 1
    
    for etiqueta in lista:
        a = etiqueta.text[:len(etiqueta.text) - 2 * len(museo) - 2].strip()
        print(a)
        print("_____________________")
        
    return True